# Inser or Upsert documents to Elastic search

In [1]:
es_base_url = 'http://localhost:9200'
headers = {'Content-Type': 'application/json'}

In [2]:
# auto sugestion with tab
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import datetime as datetime

import requests
import json

import os
os.getcwd()

'/lhome/amardj/ISM/python_code/es_scripts'

In [4]:
sys_date = datetime.datetime.today().strftime( '%y%m%d')

In [5]:
file_name = '/lhome/amardj/ISM/python_code/es_scripts/resources/backend_supportprim_data/2020-10/gen/' + '2_sp_dataset_clean_201009.csv'

In [6]:
df_csv = pd.read_csv(file_name, index_col='id')

In [7]:
df_csv.head()

,activity_1,age_1,back_nevro_pow_1,back_nevro_reflex_1,back_nevro_sens_1,back_slr_1,birth_year,bmi_1,body_main_1,childhood_1,...,walk_aid_1,weight_1,work_ability_1,work_ability_3,work_ability_w2,work_ability_w4,work_ability_w8,work_type_1,clinic_id,patient_id
id,,,,,,,,,,,,,,,,,,,,,
0,slightly reduced,22.0,NaN,NaN,NaN,NaN,1998,27.636057,hip,very good,...,no walkaid,78,10,9,9.0,9.0,NaN,NaN,adam_saso,adam_saso_38888_1998
1,slightly reduced,23.0,NaN,NaN,NaN,NaN,1997,18.339100,knee,moderate,...,no walkaid,53,6,5,3.0,3.0,7.0,NaN,adam_saso,adam_saso_39151_1997
2,not reduced,21.0,NaN,NaN,NaN,not applicable,1999,20.160700,back,good,...,no walkaid,69,8,9,NaN,NaN,NaN,work demand much walking,adam_saso,adam_saso_39178_1999
3,quite reduced,27.0,NaN,NaN,NaN,no,1993,25.217360,back,difficult,...,no walkaid,67,6,7,5.0,5.0,4.0,mostly seated,adam_saso,adam_saso_39185_1993
4,very reduced,31.0,NaN,NaN,NaN,NaN,1989,24.464602,neck,very good,...,no walkaid,65,0,2,3.0,3.0,2.0,mostly seated,adam_saso,adam_saso_39442_1989


In [8]:
js = df_csv.to_json(orient='index')

In [9]:
df1 = df_csv.fillna( value="")

In [10]:
dict_csv = df1.to_dict( orient='index')

In [11]:
json_str = json.dumps( dict_csv, allow_nan=True, ensure_ascii=False)

In [12]:
#print(json_str)

In [13]:
import codecs
import json

with codecs.open('2_sp_dataset_clean_201009.json', 'w', encoding='utf-8') as f:
    json.dump(dict_csv, f, ensure_ascii=False)

In [14]:
import datetime
datetime.datetime.now().astimezone().isoformat()

'2020-10-09T14:32:02.326648+02:00'

In [15]:
es_main_index = 'manual_patients'
es_trace_index = 'manual_patient_traces'

# 

In [16]:
def get_id_for_pid( pid, es_index):
    params = (('filter_path', 'hits.hits._source'))
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.get(f'{es_base_url}/{es_index}/_search?filter_path=hits.hits._id', headers=headers, data=data)
    response_json = response.json()
    return response_json.get('hits', {}).get('hits', [{}])[0].get('_id')

In [17]:
def get_created_date_for_pid( pid, es_index):
    params = (('filter_path', 'hits.hits._source'))
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.get(f'{es_base_url}/{es_index}/_search?filter_path=hits.hits._source.dateCreated', headers=headers, data=data)
    response_json = response.json()
    return response_json.get('hits', {}).get('hits', [{}])[0].get('_source',{}).get('dateCreated')

In [18]:
get_created_date_for_pid('adam_saso_38888_1998', 'manual_patient_traces')

In [19]:
def insert_es_new_doc(json_doc, es_index):
    doc = json.dumps(json_doc)
    response = requests.post(f'{es_base_url}/{es_index}/_doc', headers=headers, data=doc)
    return response.json()

In [20]:
def update_es_doc_for_id( _id, json_doc, es_index):
    upsert_doc = json.dumps({'doc': json.dumps(json_doc), 'doc_as_upsert': True})
    response = requests.post(f'{es_base_url}/{es_index}/_update/{_id}?pretty', headers=headers, data=upsert_doc)
    return response.json()

In [21]:
def upadate_es_index(dict_csv, es_index):     
    
    es_trace_index = 'manual_patient_traces'
    
    for val in dict_csv:

        doc_json = {}
        
        patientDetails = dict_csv[val]

        patientId = patientDetails.get('patient_id')
        clinicId = patientDetails.get('clinic_id')

        current_date_time = datetime.datetime.now().astimezone().isoformat(timespec='milliseconds')

        doc_json.update(
            {
                "patientId" : patientId, 
                "clinicId" : clinicId, 
                "dateCreated" : current_date_time,
                "dateUpdated" : current_date_time, 
                "status" : "INITIAL", 
                "similarPatients" : { }, 
                "patientDetails": patientDetails
            }
        )
        
        #print(patientId)
        _id = get_id_for_pid( patientId, es_index)
        #print(_id)
        
        
        if _id is None:
            print('_id was None, thus inserting doc for pid', patientId)
                    
            insert_es_new_doc(doc_json, es_index)
            insert_es_new_doc(doc_json, es_trace_index)
        else :
            print('Upserting doc for pid:', patientId, ' and _id: ', _id)
            
            doc_json['dateCreated'] = get_created_date_for_pid( patientId, es_index)
            
            insert_es_new_doc(doc_json, es_trace_index)
            update_es_doc_for_id(_id, doc_json, es_index)

In [22]:
upadate_es_index( dict_csv, es_main_index)

_id was None, thus inserting doc for pid adam_saso_38888_1998
_id was None, thus inserting doc for pid adam_saso_39151_1997
_id was None, thus inserting doc for pid adam_saso_39178_1999
_id was None, thus inserting doc for pid adam_saso_39185_1993
_id was None, thus inserting doc for pid adam_saso_39442_1989
_id was None, thus inserting doc for pid alette_bade_rygh_1808_1969
_id was None, thus inserting doc for pid alette_bade_rygh_1810_1989
_id was None, thus inserting doc for pid astrid_holmen_8853_1968
_id was None, thus inserting doc for pid astrid_holmen_39465_1967
_id was None, thus inserting doc for pid fredrik_paulsberg_31903_1972
_id was None, thus inserting doc for pid fredrik_paulsberg_31910_1968
_id was None, thus inserting doc for pid fredrik_paulsberg_31992_1961
_id was None, thus inserting doc for pid fredrik_paulsberg_32035_1969
_id was None, thus inserting doc for pid gaute_alstad_sunde_2748_1996
_id was None, thus inserting doc for pid gaute_alstad_sunde_8017_1973
_id

In [24]:
datetime.datetime.now().astimezone().isoformat(timespec='milliseconds')

'2020-10-09T14:26:08.689+02:00'

In [ ]:
import requests

headers = {
    'Content-Type': 'application/json',
}

data = '{  "query": {    "match_all": {}  }}'

response = requests.post('http://supportprim.idi.ntnu.no:9200/manual_patients/_search', headers=headers, data=data)
response.json()

# Elasticsearch Operations

## Code for getting a patient by patientId from the ES

# 
## Set the elasticsearch (es) index name
## 

In [20]:
es_index = 'test_patients'

# 

In [21]:
import requests

es_base_url = 'http://localhost:9200'
headers = {'Content-Type': 'application/json'}

In [22]:
def get_all_docs(es_index=es_index):
    response = requests.post(es_base_url+'/'+es_index+'/_search?pretty', headers=headers, data='{}')
    return response.json()

In [23]:
def get_doc_by_pid(pid, es_index=es_index,):
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.post(es_base_url+'/'+es_index+'/_search?pretty', headers=headers, data=data)
    return response.json()

In [24]:
result_json = get_doc_by_pid('Vidar_Guttormsen_30160_1992');

In [25]:
if result_json.get('abc', {}).get('xyz', None) is None:
    print('None')

None


In [26]:
a = None

if a is None:
    pass
elif a == False: # elif not a
    pass
elif a == True: # elif a
    pass



if not a:
    pass
elif a:
    pass


In [27]:
def get_id_for_pid( pid, es_index=es_index):
    params = (('filter_path', 'hits.hits._source'))
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.get(f'{es_base_url}/{es_index}/_search?filter_path=hits.hits._id', headers=headers, data=data)
    response_json = response.json()
    return response_json.get('hits', {}).get('hits', [{}])[0].get('_id')

In [28]:
get_id_for_pid('Vidar_Guttormsen_30160_1992')

# 

# 

## Code for getting only the list of patient ids from the ES

In [29]:
def update_es_doc_for_id( _id, json_doc, es_index=es_index):
    upsert_doc = json.dumps({'doc': json_doc, 'doc_as_upsert': True})
    response = requests.post(f'{es_base_url}/{es_index}/_update/{_id}?pretty', headers=headers, data=upsert_doc)
    return response.json()

In [30]:
def upadate_es_index(body_maps_list=init_es, es_index=es_index):
    for body_map in body_maps_list:
        pid = body_map.get('patientId')
        print(pid)
        _id = get_id_for_pid( pid, es_index)
        print(_id)
        
        if _id is None:
            print('_id was None')
        else :
            print('Upserting the pid:',pid, ' and _id: ', _id)
            update_es_doc_for_id(_id, body_map, es_index)

NameError: name 'init_es' is not defined

In [31]:
upadate_es_index( init_es, 'test_patients')

NameError: name 'upadate_es_index' is not defined